In [1]:
!pip install bs4

  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1272 sha256=962a45bea1bfa6d6228df0d2b376f0b532460283b00c3e17c8360965b6803a1d
  Stored in directory: /home/aitugan/.cache/pip/wheels/75/78/21/68b124549c9bdc94f822c02fb9aa3578a669843f9767776bca
Successfully built bs4


In [1]:
import requests
from bs4 import BeautifulSoup

In [3]:
TOPICS = {
  "economic": "Экономика",
  "culture": "Культура",
  "medicine": "Медицина",
  "science": "Наука",
  "tech": "Технологии",
  "tengri-sport": "Спорт",
  "life":"Жизнь",
  "show": "Шоу-бизнес",
  "accidents":"Происшествия",
  "crime":"Преступность",
}
URL = "https://tengrinews.kz"


In [3]:
def parse_links(url, topic, pages_num=50):
  path = '{}/{}'.format(url,topic)
  articles_links = {}
  for page in range(1, pages_num+1):
    response = requests.get("{}/page/{}".format(path, str(page)))

    if response.status_code == 200:
      soup = BeautifulSoup(response.content, 'html.parser')

      article_divs = soup.find_all("div", "tn-article-item")

      for article in article_divs:
        a_tag = article.find('a')

        title = article.find("span", "tn-article-title")
        if title:
          title = title.get_text()

        if a_tag:
          link = url + a_tag['href']
          articles_links[title] = link

    else:
      print(f"Failed to scrape the website. Status code: {response.status_code}")
  return articles_links

In [4]:
import os
def save_article_to_file(title, article, topic):
  directory = f'../articles/{topic}'
  os.makedirs(directory, exist_ok=True)
  title = title.replace('"', '')
  title = title.replace('/', '_')
  file_path = os.path.join(directory, f'{title}.txt')
  with open(file_path, 'w') as file:
      file.write(article)

In [5]:
def parse_articles_by_topic(topic):
  links = parse_links(URL, topic)
  for title, link in links.items():
    response = requests.get(link)
    if response.status_code == 200:
      soup = BeautifulSoup(response.content, 'html.parser')
      article_content = soup.find("div", "tn-news-content")
      if article_content is None:
        print(link)
        continue
      p_tags = article_content.find_all("p")
      article = []
      for i in range(len(p_tags)-2):
        article.append(p_tags[i].get_text())
      save_article_to_file(title, "".join(article), topic)


In [18]:
for topic in TOPICS:
  if topic == "tengri-sport": continue
  parse_articles_by_topic(topic)



In [ ]:
import os

def delete_files_in_folder(folder_path):
    file_list = os.listdir(folder_path)

    for file_name in file_list:
        file_path = os.path.join(folder_path, file_name)
        if os.path.isfile(file_path):
            os.remove(file_path)
            print(f"Deleted: {file_path}")


# Parser for sport articles

In [7]:
def parse_sport_articles(topic):
  links = parse_links(URL, topic)
  for title, link in links.items():
    response = requests.get(link)
    if response.status_code == 200:
      soup = BeautifulSoup(response.content, 'html.parser')
      article_content = soup.find("div", "news__text")
      if article_content is None:
        print(link)
        continue
      p_tags = article_content.find_all("p")
      article = []
      for i in range(len(p_tags)-1):
        article.append(p_tags[i].get_text())
      save_article_to_file(title, "".join(article), topic)

parse_sport_articles("tengri-sport")